In [2]:
import pandas as pd
from functools import partial
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict, Counter

**Load nucleolar protein data with localCIDER and NARDINI scores**

In [4]:
representations_45S_47S_df_w_cider_nardini = pd.read_csv("data/curated_data_for_modeling/45S_47S_localcider_features_w_nardini.tsv", sep="\t")
representations_45S_47S_df_w_cider_nardini

,Majority.protein.IDs,Reviewed,Protein ID,Sequence,Label,Length,length,FCR,NCPR,isoelectric_point,...,zsc_PROLINE_PROLINE,zsc_PROLINE_GLYCINE,zsc_GLYCINE_POLAR,zsc_GLYCINE_HYDROPHOBIC,zsc_GLYCINE_POSITIVE,zsc_GLYCINE_NEGATIVE,zsc_GLYCINE_AROMATIC,zsc_GLYCINE_ALANINE,zsc_GLYCINE_PROLINE,zsc_GLYCINE_GLYCINE
0,A0A2I3BPG9;P83882,reviewed,P83882,MVNVPKTRRTFCKKCGKHQPHKVTQYKKGKDSLYAQGKRRYDRKQS...,S47,106,106,0.367925,0.254717,10.71875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B2RY56,reviewed,B2RY56,MSFPPHLNRPPMGIPALPPGIPPPQFPGFPPPVPPGTPMIPVPMSI...,S45,838,838,0.478520,-0.010740,7.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E9Q8D0,reviewed,E9Q8D0,MKCHYEALGVRRDASEEELKKAYRKLALRWHPDKNLDNAAEAAEQF...,S47,531,531,0.386064,-0.024482,5.25000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,O08528,reviewed,O08528,MIASHMIACLFTELNQNQVQKVDQYLYHMRLSDETLLEISRRFRKE...,S47,917,917,0.272628,-0.019629,6.12500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,O08810,reviewed,O08810,MDTDLYDEFGNYIGPELDSDEDDDELGRETKDLDEDEDEDEDDVGE...,S45,971,971,0.263646,-0.057673,4.59375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Q9WVA4,reviewed,Q9WVA4,MANRGPSYGLSREVQQKIEKQYDADLEQILIQWITTQCREDVGQPQ...,S45,199,199,0.221106,0.010050,8.75000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,Q9Z0U1,reviewed,Q9Z0U1,MEEVIWEQYTVTLQKDSKRGFGIAVSGGRDNPHFENGETSIVISDV...,S47,1167,1167,0.309340,-0.007712,7.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,Q9Z199;P63271,reviewed,Q9Z199,MALETVPKDLRHLRACLLCSLVKTIDQFEYDGCDNCDAYLQMKGNR...,S47,117,117,0.256410,0.017094,8.31250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,Q9Z1N5,reviewed,Q9Z1N5,MAENDVDNELLDYEDDEVETAAGADGTEAPAKKDVKGSYVSIHSSG...,S45,428,428,0.282710,-0.030374,5.25000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Import the mouse proteome from Uniprot**

In [7]:
uniprot_df = pd.read_csv("data/uniprot_data/uniprotkb_taxonomy_id_10090_2025_03_25.tsv", sep="\t")   
uniprot_df

/tmp/ipykernel_558039/3283731104.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  uniprot_df = pd.read_csv("/ceph/hpc/home/novljanj/data_storage/projects/nucleolus_enriched_proteins/data/uniprot_data/uniprotkb_taxonomy_id_10090_2025_03_25.tsv", sep="\t")


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Compositional bias,Mass,Sequence,...,Redox potential,Region,Ensembl,Coiled coil,Domain [CC],Domain [FT],Motif,Repeat,Zinc finger,Gene Ontology (cellular component)
0,A0A075F5C6,unreviewed,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),531,"COMPBIAS 343..355; /note=""Polar residues""; /ev...",57879,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,...,NaN,"REGION 272..327; /note=""Disordered""; /evidence...",ENSMUST00000228371.2;,NaN,NaN,"DOMAIN 57..81; /note=""HSF-type DNA-binding""; /...",NaN,NaN,NaN,centrosome [GO:0005813]; cytosol [GO:0005829];...
1,A0A087WPF7,reviewed,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),1261,"COMPBIAS 8..17; /note=""Basic residues""; /evide...",138920,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,...,NaN,"REGION 1..88; /note=""Disordered""; /evidence=""E...",ENSMUST00000161226.11 [A0A087WPF7-1];ENSMUST00...,NaN,DOMAIN: The Pro-rich region is important for t...,NaN,NaN,NaN,NaN,cytoplasm [GO:0005737]; cytoskeleton [GO:00058...
2,A0A087WPT2,unreviewed,A0A087WPT2_MOUSE,Prostaglandin-endoperoxide synthase 2,Ptgs2,Mus musculus (Mouse),62,NaN,7199,MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYIL...,...,NaN,NaN,ENSMUST00000190784.2;,NaN,NaN,"DOMAIN 1..23; /note=""EGF-like""; /evidence=""ECO...",NaN,NaN,NaN,NaN
3,A0A087WRK1,unreviewed,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),222,NaN,25620,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,...,NaN,"REGION 39..64; /note=""Disordered""; /evidence=""...",ENSMUST00000185240.2;ENSMUST00000185245.2;ENSM...,NaN,NaN,"DOMAIN 112..220; /note=""XLR/SYCP3/FAM9""; /evid...",NaN,NaN,NaN,cytoplasm [GO:0005737]; nucleus [GO:0005634]
4,A0A087WRT4,unreviewed,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),4602,"COMPBIAS 4258..4267; /note=""Polar residues""; /...",507531,MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,...,NaN,"REGION 4256..4292; /note=""Disordered""; /eviden...",ENSMUST00000189017.8;,NaN,NaN,"DOMAIN 36..150; /note=""Cadherin""; /evidence=""E...",NaN,NaN,NaN,apical plasma membrane [GO:0016324]; cell junc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87487,Z4YN82,unreviewed,Z4YN82_MOUSE,Syntaphilin,Snph,Mus musculus (Mouse),87,NaN,9364,MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGS...,...,NaN,"REGION 1..74; /note=""Disordered""; /evidence=""E...",ENSMUST00000137936.2;,NaN,NaN,NaN,NaN,NaN,NaN,membrane [GO:0016020]
87488,Z4YN86,unreviewed,Z4YN86_MOUSE,Translocase of inner mitochondrial membrane 22,Timm22,Mus musculus (Mouse),116,NaN,12689,MAATAPKAGGSAPEAAGSAEAPLQYSLLLQYLVGDKRQPRLLEPGS...,...,NaN,NaN,ENSMUST00000152183.2;,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87489,Z4YN92,unreviewed,Z4YN92_MOUSE,A kinase anchor protein 17B,Akap17b,Mus musculus (Mouse),406,"COMPBIAS 24..52; /note=""Basic and acidic resid...",47765,MFDTTKHFSEGAIQRRNQERLKLQELEEERKKEKKREEEVAERKRK...,...,NaN,"REGION 24..90; /note=""Disordered""; /evidence=""...",ENSMUST00000133980.2;,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87490,Z4YN97,unreviewed,Z4YN97_MOUSE,Adenylate kinase 1,Ak1,Mus musculus (Mouse),89,NaN,9541,MEEKLKKAKIIFVVGGPGSGKGTQCEKIVQKYGYTHLSTGDLLRAE...,...,NaN,NaN,ENSMUST00000156578.8;,NaN,NaN,NaN,NaN,NaN,NaN,cytoplasm [GO:0005737]


**Parse the domains, regions and compositional bias annotation over protein sequence into the percent of protein**

In [8]:
representations_45S_47S_df_w_cider_nardini_with_uniprot = representations_45S_47S_df_w_cider_nardini.merge(uniprot_df, left_on="Protein ID", right_on="Entry", how="inner")

import pandas as pd, re

FEATURE_RE  = re.compile(r'(\d+)\.\.(\d+).*?/note="([^"]+)"')  
NOTE_CLEAN  = re.compile(r'\s*\d+$')                          

def feature_matrix(df: pd.DataFrame,
                   feature_col: str = "Domain [FT]",
                   length_col: str  = "Length_x") -> pd.DataFrame:

    unique_notes = set()
    for cell in df[feature_col].fillna(""):
        for _, _, raw_note in FEATURE_RE.findall(cell):
            unique_notes.add(NOTE_CLEAN.sub("", raw_note).strip())
    note_list = sorted(unique_notes)

    matrix = pd.DataFrame(0.0, index=df.index, columns=note_list)

    for idx, row in df[[feature_col, length_col]].iterrows():
        cell, prot_len = row[feature_col], row[length_col]

        if pd.isna(cell) or cell == "" or pd.isna(prot_len):
            continue

        prot_len = float(prot_len)

        for start, end, raw_note in FEATURE_RE.findall(cell):
            span       = int(end) - int(start) + 1
            clean_note = NOTE_CLEAN.sub("", raw_note).strip()
            matrix.at[idx, clean_note] += span / prot_len

    return matrix

with_uni = representations_45S_47S_df_w_cider_nardini_with_uniprot
base     = representations_45S_47S_df_w_cider_nardini

for col in ["Domain [FT]", "Region", "Compositional bias"]:
    matrix = feature_matrix(with_uni, feature_col=col)
    base = pd.concat([base, matrix], axis=1)

representations_45S_47S_df_w_nardini_with_uniprot_parsed = base 

print(f"Added {len(representations_45S_47S_df_w_nardini_with_uniprot_parsed.columns.difference(representations_45S_47S_df_w_cider_nardini.columns))} columns to the dataframe")

representations_45S_47S_df_w_nardini_with_uniprot_parsed

Added 216 columns to the dataframe


,Majority.protein.IDs,Reviewed,Protein ID,Sequence,Label,Length,length,FCR,NCPR,isoelectric_point,...,Transcription repression,Trigger loop,VCA,WHD,immunoglobulin-like,Acidic residues,Basic and acidic residues,Basic residues,Polar residues,Pro residues
0,A0A2I3BPG9;P83882,reviewed,P83882,MVNVPKTRRTFCKKCGKHQPHKVTQYKKGKDSLYAQGKRRYDRKQS...,S47,106,106,0.367925,0.254717,10.71875,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,B2RY56,reviewed,B2RY56,MSFPPHLNRPPMGIPALPPGIPPPQFPGFPPPVPPGTPMIPVPMSI...,S45,838,838,0.478520,-0.010740,7.00000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.250597,0.000000,0.011933,0.022673
2,E9Q8D0,reviewed,E9Q8D0,MKCHYEALGVRRDASEEELKKAYRKLALRWHPDKNLDNAAEAAEQF...,S47,531,531,0.386064,-0.024482,5.25000,...,0.0,0.0,0.0,0.0,0.0,0.018832,0.067797,0.060264,0.039548,0.000000
3,O08528,reviewed,O08528,MIASHMIACLFTELNQNQVQKVDQYLYHMRLSDETLLEISRRFRKE...,S47,917,917,0.272628,-0.019629,6.12500,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,O08810,reviewed,O08810,MDTDLYDEFGNYIGPELDSDEDDDELGRETKDLDEDEDEDEDDVGE...,S45,971,971,0.263646,-0.057673,4.59375,...,0.0,0.0,0.0,0.0,0.0,0.031926,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Q9WVA4,reviewed,Q9WVA4,MANRGPSYGLSREVQQKIEKQYDADLEQILIQWITTQCREDVGQPQ...,S45,199,199,0.221106,0.010050,8.75000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
198,Q9Z0U1,reviewed,Q9Z0U1,MEEVIWEQYTVTLQKDSKRGFGIAVSGGRDNPHFENGETSIVISDV...,S47,1167,1167,0.309340,-0.007712,7.00000,...,0.0,0.0,0.0,0.0,0.0,0.011140,0.068552,0.000000,0.000000,0.000000
199,Q9Z199;P63271,reviewed,Q9Z199,MALETVPKDLRHLRACLLCSLVKTIDQFEYDGCDNCDAYLQMKGNR...,S47,117,117,0.256410,0.017094,8.31250,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
200,Q9Z1N5,reviewed,Q9Z1N5,MAENDVDNELLDYEDDEVETAAGADGTEAPAKKDVKGSYVSIHSSG...,S45,428,428,0.282710,-0.030374,5.25000,...,0.0,0.0,0.0,0.0,0.0,0.044393,0.000000,0.000000,0.000000,0.000000


In [9]:
representations_45S_47S_df_w_nardini_with_uniprot_parsed.to_csv("data/curated_data_for_modeling/45S_47S_localcider_features_w_nardini_w_uniprot.tsv", sep="\t", index=False)